# Evaluating definite integrals using `TaylorIntegration.jl`

The aim of this example is to show that we can evaluate definite integrals of the form 

$$
I(x) = \int_{x_0}^xf(t)\mathrm{d} t
$$

for a large class of differentiable integrand functions $f$, to within machine epsilon precision, using `TaylorIntegration.jl`. The trick is to translate the definite integral problem to an initial value problem for an explicit ODE, thanks to the fundamental theorem of calculus.

## Error function: $I(x)=\frac{2}{\sqrt{\pi}}\int_0^x \exp(-t^2) dt$

Here, we want to evaluate the [__error function__](https://en.wikipedia.org/wiki/Error_function), defined by the integral $\operatorname{erf}(x)=\frac{2}{\sqrt{\pi}}\int_0^x \exp(-t^2) dt$ using `TaylorIntegration.jl`. We proceed as follows:

First, define a suitable ODE system equivalent to the definite integral problem at hand, and then integrate it. Indeed, by the fundamental theorem of calculus we have

$$
\operatorname{erf}'(x) =\frac{2}{\sqrt{\pi}}\exp(-x^2)
$$


where the prime $'$ denotes differentiation with respect to $x$. Therefore, we reinterpret $I(x)$ as the solution of the initial value problem defined by the ODE

$$
\frac{d\operatorname{erf}}{dx} = \frac{2}{\sqrt{\pi}}\exp(-x^2)
$$

subject to the initial condition $\operatorname{erf}(0)=0$. We will integrate this ODE in order to evaluate $\operatorname{erf}(x)$.

The only packages that we need to load are:

In [1]:
using TaylorIntegration, Plots
gr()

Plots.GRBackend()

This is the ODE representation of the integral we are interested in (below, `t` is the independent variable):

In [2]:
f(t, x) = (2/sqrt(pi))*exp(-t^2)

f (generic function with 1 method)

The parameters we will use for the Taylor integration are:

In [3]:
x0 = 0.0 #the initial value of the independent variable; in this case, x
xmax = 10.0 #the final value of the independent variable; in this case, x
erf0 = 0.0 #the initial condition for erf
order = 25 #the order of the Taylor expansions
abstol = 1e-20 #the absolute local error tolerance

1.0e-20

The initial conditions are:

In [4]:
x0, erf0

(0.0, 0.0)

Then, we proceed to integrate:

In [5]:
@time xv, erfv = taylorinteg(f, erf0, x0, xmax, order, abstol, maxsteps=5000);

  0.401981 seconds (170.16 k allocations: 9.200 MiB)


The final $x$ and $\operatorname{erf}$ values are:

In [6]:
xv[end], erfv[end]

(10.0, 1.0)

Now, how does $\operatorname{erf}(x)$ look as a function of $x$?

In [9]:
plot(xv, erfv)
scatter!(xv, erfv, ms=3.0)

xlabel!("x")
ylabel!("erf(x)")
title!("Evaluating erf(x) using TaylorIntegration")

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0.0 
 
 
 2.5 
 
 
 5.0 
 
 
 7.5 
 
 
 10.0 
 
 
 0.00 
 
 
 0.25 
 
 
 0.50 
 
 
 0.75 
 
 
 1.00 
 
 
 Evaluating erf(x) using TaylorIntegration 
 
 
 x 
 
 
 erf(x) 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 y1 
 
 
 
 
 y2

At a first glance, thsi looks great! But, how do these values actually compare to Julia's `Base.erf`?

In [13]:
?Base.erf

```
erf(x)
```

Compute the error function of `x`, defined by $\frac{2}{\sqrt{\pi}} \int_0^x e^{-t^2} dt$ for arbitrary complex `x`.


Well, let's plot the absolute difference between our integrated values for $\operatorname{erf}(x)$ and `Base.erf`, in units of the machine epsilon, as a function of $x$:

In [14]:
plot(

xv, (erfv-erf.(xv))/eps(),
xaxis = "x",
yaxis = "Diff: integrated vs Base.erf",
title = "Absolute difference between erf(x) and I(x)"

)

scatter!(xv, (erfv-erf.(xv))/eps(), ms=3.0)

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0.0 
 
 
 2.5 
 
 
 5.0 
 
 
 7.5 
 
 
 10.0 
 
 
 0.0 
 
 
 0.1 
 
 
 0.2 
 
 
 0.3 
 
 
 0.4 
 
 
 0.5 
 
 
 Absolute difference between erf(x) and I(x) 
 
 
 x 
 
 
 Diff: integrated vs Base.erf 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 y1 
 
 
 
 
 y2

### The error is at most half times `eps()` at each time step!!!!

## Evaluating elliptic integrals with `BigFloats`

In this section, we will evaluate the elliptic integral $K$ using `BigFloats`, up to an accuracy of about $10^{-77}$ (!)

In [16]:
# work in progress...